In [1]:
import time
import math
import random
import itertools as its
import numpy as np
import pandas as pd
import tqdm

from collections import Counter

In [2]:
dt = pd.read_csv("../data/user2up.csv")
dt.rename(columns={"mid":"up","from":"user"},inplace=True)
dt.up = dt.up.astype(int)
dt_fmt = dt[["user","up"]]

In [3]:
dt_fmt

,user,up
0,81922,326499679
1,81922,316568752
2,81922,9617619
3,81922,213741
4,81922,676498418
...,...,...
697970,519570206,456664753
697971,519570206,14114387
697972,519570206,504739753
697973,519570206,414021604


In [4]:
class UserCF():
    def __init__(self,data,K,N,split_n=0.7,ignore_user_n=10,ignore_up_n=10,seed=123):
        self.data = data
        self.K = K
        self.N = N
        self.split_n = split_n
        self.ignore_user_n = ignore_user_n
        self.ignore_up_n = ignore_up_n
        self.seed = seed
        self.data_format(data)

        
    def data_format(self,data):
        

        # 为了保证在切分用户行为数据的时候,训练与测试集均保证一定数量
        # 此处过滤掉行为数量过少的UP
        up_behavior_count = data.groupby("up").count()
        ups = set(up_behavior_count[up_behavior_count.user>=self.ignore_up_n].index)
        
        user_behavior_count = data.groupby("user").count()
        users = set(user_behavior_count[user_behavior_count.up>=self.ignore_user_n].index)
        
        # 此处过滤掉行为数量过少的User和UP
        data = data[(data.up.isin(ups)&(data.user.isin(users)))]
        
        # 此处根据过滤后的结果重新获取集合
        users = set(data.user.unique())
        ups = set(data.up.unique())

        data = data.sample(frac=1,random_state=self.seed)

#         # 全随机切分
#         split = int(len(data)*self.split_n)
#         train_df = data[:split]
#         test_df = data[split:]
#         users = train_df.user.unique()
#         ups = test_df.up.unique()
        
        # 基于单用户内的关注行为切分
        train_df = pd.DataFrame()
        test_df = pd.DataFrame()
        for user_tmp in tqdm.tqdm(data.user.unique(),"切割数据"):
            data_user = data[data.user==user_tmp]
            data_user_split = int(len(data_user)*self.split_n)
            train_df = train_df.append(data_user[:data_user_split])
            test_df = test_df.append(data_user[data_user_split:])
        
        
#         # User对UP主的关注集
#         # 切分为训练测试集
#         matrix_user2up = train_df.groupby("user").agg(set)
#         matrix_user2up["train"] = matrix_user2up["up"].apply(self.sample,args=(self.split_n,self.seed))
#         matrix_user2up[["train","test"]] = matrix_user2up["train"].apply(pd.Series)
        # User对UP主的关注集
        # 切分为训练测试集
        matrix_user2up = data.groupby("user").agg(set)
        matrix_user2up["train"] = train_df.groupby("user").agg(set)
        matrix_user2up["test"] = test_df.groupby("user").agg(set)

        
        # UP主对User倒排表
        matrix_up2user = train_df.groupby("up").agg(set)
        matrix_up2user = matrix_up2user.user.to_dict()
        
        train = matrix_user2up.train.to_dict()
        test = matrix_user2up.test.to_dict()
        

        for user in train:
            if train[user] is np.nan:
                train[user] = set()
        
        for user in test:
            if test[user] is np.nan:
                test[user] = set()
        
        # 倒排表matrix_up2user降低计算量
        matrix_user2user = {}
        count = 0
        
        # 遍历倒排表,对存在关注同一个up主的user进行相似度计算
        # TODO 此处考虑使用numpy来做一遍或许更有利于避免重复计算
        for up,sub_users in tqdm.tqdm(matrix_up2user.items(),"用户间相似度"):
            for user_a,user_b in its.combinations(sub_users,2):
                # 排序限定为左小右大
                user_a,user_b = (user_a,user_b) if user_a < user_b else (user_b,user_a)
                # 若无历史结果,则计算
                if matrix_user2user.get((user_a,user_b)) == None:
                    # TODO,应该支持多类相似度算法
                    # 此处为快速实现使用了杰卡德相似度
                    matrix_user2user[(user_a,user_b)] = len(train[user_a]&train[user_b])/len(train[user_a]|train[user_b])
                count += 1
        print("倒排表约使计算量减少至原来的%s倍."%round(count/(len(users)**2),2))
        
        self.data = data
        self.users = users
        self.ups = ups
        self.matrix_user2up = matrix_user2up
        self.matrix_up2user = matrix_up2user
        self.matrix_user2user = matrix_user2user
        self.train = train
        self.test = test
        
#     def sample(self,up_set,train_k,seed):
#         random.seed(seed)
#         train = set(random.sample(up_set,int(len(up_set)*train_k)))
#         test = up_set - train
#         return train,test
    
    def GetRecommendation(self,user):
        user2user = {}
        
        # 计算目标user与其他user的相似度
        for user_other in self.users:
            if user_other == user:
                continue
            # 标准化key
            user_a,user_b = (user,user_other) if user < user_other else (user_other,user)
            user2user[user_other] = self.matrix_user2user.get((user_a,user_b),0)
        
        # TODO 此处的推荐过程可以增加缓存,这样子在仅变动K,N的时候,不用重复计算topK_user和topN_up
        # 根据user2user的value相似度降序排后取前K位
        topK_user = sorted(user2user.items(),key=lambda x: x[1],reverse=True)[:self.K]
        topK_user_ups_list = []
        
        # 取topK_user的关注列表
        for user_other,score in topK_user:
            topK_user_ups_list += self.train[user_other]
        # 注意去掉当前已推荐的
        counter = Counter(topK_user_ups_list)
        counter = {up:n for up,n in counter.items() if up not in self.train[user]}
        topN_up = sorted(counter.items(),key=lambda x: x[1],reverse=True)[:self.N]
        return set(topN_up)

In [5]:
class EvaluationMetrics():
    def __init__(self,train,test,get_rec):
        """
        :param train: {123:{1,2,3,4,5}}
        :param test: {123:{1,2,3,4,5}}
        :param get_rec: recs = get_rec(user)
        """
        self.train = train
        self.test = test
        self.get_rec = get_rec
        self.recs = self.get_rec_all_user()
        self.hit = self._hit()
    
    # 获得所有test用户的推荐结果
    def get_rec_all_user(self):
        # recs = {1:{(64554310,9),(123142,8)}}
        recs = {}
        for user in tqdm.tqdm(self.train,"推荐中"):
            recs[user] = self.get_rec(user)
        return recs
    
    # 分子/命中
    def _hit(self):
        hit = 0
        for user in self.test:
            sub_test = self.test[user]
            rec = {item for item,score in self.recs[user]}
            hit += len(sub_test&rec)
        return hit
    
    # 精确率
    def precision(self):
        count = 0
        for user in self.test:
            count += len(self.recs[user])
        return round(self.hit / count * 100,2)
    
    # 召回率
    def recall(self):
        count = 0
        for user in self.test:
            count += len(self.test[user])
        return round(self.hit / count * 100,2)
    
    # 覆盖率
    def coverage(self):
        all_items = set()
        rec_items = set()
        for user,items in self.train.items():
            all_items = all_items|items
        # TODO 这边考虑直接以模型的data.ups.unique()去处理避免重复计算
        for user,items in self.recs.items():
            rec_items = rec_items|set([item for item,score in items])
        return round(len(rec_items) / len(all_items) * 100,2)
    
    # 新颖度
    def popularity(self):
        # 计算物品的流行度
        item_pop = {}
        for user in self.train:
            for item in self.train[user]:
                if item not in item_pop:
                    item_pop[item] = 0
                item_pop[item] += 1

        num, pop = 0, 0
        for user in self.test:
            rank = self.recs[user]
            for item, score in rank:
                # 取对数
                pop += math.log(1 + item_pop[item])
                num += 1
        return round(pop / num, 6)
    
    def evaluate(self):
        evaluation_metrics = {'precision': self.precision(),
                              'recall': self.recall(),
                              'coverage': self.coverage(),
                              'popularity': self.popularity()}
        print('EvaluationMetrics:', evaluation_metrics)
        return evaluation_metrics

In [6]:
class Experiment():
    def __init__(self,model,T):
        self.model = model
        self.T = T
    
    def validate(self):
        resulst = EvaluationMetrics(self.model.train,self.model.test,self.model.GetRecommendation)
        return resulst.evaluate()

In [7]:
results = pd.DataFrame(columns=["model","split_n","K","N",
                                "ignore_user_n","ignore_up_n","seed",
                                "precision","recall","coverage","popularity"])

In [8]:
dt_fmt.groupby("user").agg(len).describe()

,up
count,10009.000000
mean,69.734739
std,49.848071
min,1.000000
25%,26.000000
50%,85.000000
75%,100.000000
max,500.000000


In [9]:
for ignore_user_n,ignore_up_n,seed in its.product([10],[100,10],[1,2,3]):
    model_usercf = UserCF(dt_fmt,K=30,N=30,split_n=0.7,
                          ignore_user_n=ignore_user_n,ignore_up_n=ignore_up_n,seed=seed)
    for K,N in its.product([10,50,100,500],[20,50,100]):
        print(ignore_user_n,ignore_up_n,seed,
              K,N)
        time.sleep(0.5)
        model_usercf.K = K
        model_usercf.N = N
        usercf_exp = Experiment(model_usercf,5)

        res = usercf_exp.validate()
        res["model"] = "UserCF"
        res["split_n"] = 0.7
        res["ignore_user_n"] = ignore_user_n
        res["ignore_up_n"] = ignore_up_n
        res["seed"] = seed
        res["K"] = K
        res["N"] = N
        results = results.append(res,ignore_index=True)

用户间相似度: 100%|██████████████████████████████████████████████████████████████████| 619/619 [00:24<00:00, 25.23it/s]


倒排表约使计算量减少至原来的0.19倍.
10 100 1 10 20


推荐中: 100%|█████████████████████████████████████████████████████████████████████| 8427/8427 [01:22<00:00, 102.06it/s]


EvaluationMetrics: {'precision': 5.08, 'recall': 16.21, 'coverage': 100.0, 'popularity': 5.38547}
10 100 1 10 50


推荐中: 100%|█████████████████████████████████████████████████████████████████████| 8427/8427 [01:22<00:00, 102.11it/s]


...

EvaluationMetrics: {'precision': 6.13, 'recall': 8.91, 'coverage': 3.18, 'popularity': 6.305945}
10 10 3 500 50


推荐中: 100%|██████████████████████████████████████████████████████████████████████| 8675/8675 [02:06<00:00, 68.74it/s]


EvaluationMetrics: {'precision': 4.08, 'recall': 14.82, 'coverage': 8.09, 'popularity': 5.957953}
10 10 3 500 100


推荐中: 100%|██████████████████████████████████████████████████████████████████████| 8675/8675 [02:07<00:00, 67.98it/s]


EvaluationMetrics: {'precision': 3.0, 'recall': 21.82, 'coverage': 17.17, 'popularity': 5.647832}


In [11]:
results.sort_values("precision")

,model,split_n,K,N,ignore_user_n,ignore_up_n,seed,precision,recall,coverage,popularity
5,UserCF,0.7,50,100,10,100,1,2.84,48.51,100.00,5.273921
29,UserCF,0.7,50,100,10,100,3,2.85,48.67,100.00,5.277413
17,UserCF,0.7,50,100,10,100,2,2.85,48.59,100.00,5.279771
62,UserCF,0.7,10,100,10,10,3,2.86,18.47,99.94,4.059773
38,UserCF,0.7,10,100,10,10,1,2.87,18.64,99.98,4.064063
...,...,...,...,...,...,...,...,...,...,...,...
42,UserCF,0.7,100,20,10,10,1,7.36,10.69,21.15,5.916473
54,UserCF,0.7,100,20,10,10,2,7.43,10.79,21.51,5.918746
63,UserCF,0.7,50,20,10,10,3,7.55,10.97,40.61,5.595071
39,UserCF,0.7,50,20,10,10,1,7.56,10.99,40.30,5.599809


In [13]:
results.K.unique()

array([10, 50, 100, 500], dtype=object)

In [14]:
results.N.unique()

array([20, 50, 100], dtype=object)

In [12]:
results[results.precision>=7]

,model,split_n,K,N,ignore_user_n,ignore_up_n,seed,precision,recall,coverage,popularity
39,UserCF,0.7,50,20,10,10,1,7.56,10.99,40.30,5.599809
42,UserCF,0.7,100,20,10,10,1,7.36,10.69,21.15,5.916473
51,UserCF,0.7,50,20,10,10,2,7.63,11.09,41.57,5.595002
54,UserCF,0.7,100,20,10,10,2,7.43,10.79,21.51,5.918746
63,UserCF,0.7,50,20,10,10,3,7.55,10.97,40.61,5.595071
66,UserCF,0.7,100,20,10,10,3,7.32,10.64,21.17,5.917593


In [15]:
for ignore_user_n,ignore_up_n,seed in its.product([10],[10],[1,2,3]):
    model_usercf = UserCF(dt_fmt,K=30,N=30,split_n=0.7,
                          ignore_user_n=ignore_user_n,ignore_up_n=ignore_up_n,seed=seed)
#     for K,N in its.product([10,50,100,500],[20,50,100]):
    for K,N in its.product([30,50,75,100],[25,40]):
        print(ignore_user_n,ignore_up_n,seed,
              K,N)
        time.sleep(0.5)
        model_usercf.K = K
        model_usercf.N = N
        usercf_exp = Experiment(model_usercf,5)

        res = usercf_exp.validate()
        res["model"] = "UserCF"
        res["split_n"] = 0.7
        res["ignore_user_n"] = ignore_user_n
        res["ignore_up_n"] = ignore_up_n
        res["seed"] = seed
        res["K"] = K
        res["N"] = N
        results = results.append(res,ignore_index=True)

用户间相似度: 100%|█████████████████████████████████████████████████████████████| 10778/10778 [00:41<00:00, 260.88it/s]


倒排表约使计算量减少至原来的0.21倍.
10 10 1 30 25


推荐中: 100%|██████████████████████████████████████████████████████████████████████| 8675/8675 [01:33<00:00, 92.92it/s]


EvaluationMetrics: {'precision': 6.73, 'recall': 12.21, 'coverage': 65.04, 'popularity': 5.205999}
10 10 1 30 40


推荐中: 100%|██████████████████████████████████████████████████████████████████████| 8675/8675 [01:32<00:00, 93.40it/s]


...

EvaluationMetrics: {'precision': 5.53, 'recall': 16.07, 'coverage': 50.9, 'popularity': 5.47657}
10 10 3 100 25


推荐中: 100%|██████████████████████████████████████████████████████████████████████| 8675/8675 [01:36<00:00, 90.17it/s]


EvaluationMetrics: {'precision': 6.71, 'recall': 12.18, 'coverage': 26.52, 'popularity': 5.816454}
10 10 3 100 40


推荐中: 100%|██████████████████████████████████████████████████████████████████████| 8675/8675 [01:36<00:00, 90.33it/s]


EvaluationMetrics: {'precision': 5.47, 'recall': 15.89, 'coverage': 41.04, 'popularity': 5.601536}


In [16]:
for ignore_user_n,ignore_up_n,seed in its.product([10],[10],[1,2,3]):
    model_usercf = UserCF(dt_fmt,K=30,N=30,split_n=0.7,
                          ignore_user_n=ignore_user_n,ignore_up_n=ignore_up_n,seed=seed)
#     for K,N in its.product([10,50,100,500],[20,50,100]):
    for K,N in its.product([30,50,75,100],[10,15]):
        print(ignore_user_n,ignore_up_n,seed,
              K,N)
        time.sleep(0.5)
        model_usercf.K = K
        model_usercf.N = N
        usercf_exp = Experiment(model_usercf,5)

        res = usercf_exp.validate()
        res["model"] = "UserCF"
        res["split_n"] = 0.7
        res["ignore_user_n"] = ignore_user_n
        res["ignore_up_n"] = ignore_up_n
        res["seed"] = seed
        res["K"] = K
        res["N"] = N
        results = results.append(res,ignore_index=True)

用户间相似度: 100%|█████████████████████████████████████████████████████████████| 10778/10778 [00:41<00:00, 262.09it/s]


倒排表约使计算量减少至原来的0.21倍.
10 10 1 30 10


推荐中: 100%|██████████████████████████████████████████████████████████████████████| 8675/8675 [01:31<00:00, 95.00it/s]


EvaluationMetrics: {'precision': 9.73, 'recall': 7.07, 'coverage': 35.54, 'popularity': 5.618514}
10 10 1 30 15


推荐中: 100%|██████████████████████████████████████████████████████████████████████| 8675/8675 [01:31<00:00, 95.22it/s]


...

EvaluationMetrics: {'precision': 8.42, 'recall': 9.17, 'coverage': 21.64, 'popularity': 5.923604}
10 10 3 100 10


推荐中: 100%|██████████████████████████████████████████████████████████████████████| 8675/8675 [01:38<00:00, 88.01it/s]


EvaluationMetrics: {'precision': 9.65, 'recall': 7.01, 'coverage': 10.79, 'popularity': 6.211846}
10 10 3 100 15


推荐中: 100%|██████████████████████████████████████████████████████████████████████| 8675/8675 [01:38<00:00, 88.21it/s]


EvaluationMetrics: {'precision': 8.28, 'recall': 9.02, 'coverage': 15.98, 'popularity': 6.042135}


In [17]:
results.to_excel("../data/ExperimentResults.xlsx")

In [18]:
results.

,model,split_n,K,N,ignore_user_n,ignore_up_n,seed,precision,recall,coverage,popularity
0,UserCF,0.7,10,20,10,100,1,5.08,16.21,100.00,5.385470
1,UserCF,0.7,10,50,10,100,1,3.96,25.60,100.00,5.257789
2,UserCF,0.7,10,100,10,100,1,3.62,32.54,100.00,5.205964
3,UserCF,0.7,50,20,10,100,1,6.46,23.00,100.00,5.786311
4,UserCF,0.7,50,50,10,100,1,4.08,35.93,100.00,5.478618
...,...,...,...,...,...,...,...,...,...,...,...
115,UserCF,0.7,50,15,10,10,3,8.48,9.24,32.20,5.725873
116,UserCF,0.7,75,10,10,10,3,9.82,7.14,14.44,6.103686
117,UserCF,0.7,75,15,10,10,3,8.42,9.17,21.64,5.923604
118,UserCF,0.7,100,10,10,10,3,9.65,7.01,10.79,6.211846
